In [1]:
require 'torch';
require 'nn';
require 'loadcaffe';
require 'hdf5';

In [2]:
params = {caffe_model = '/home/fanfanda/style_transfer/fanfanda_neuralImages/NeuralImageSynthesis/Models/VGG_ILSVRC_19_layers_conv.caffemodel', 
    images = '/home/fanfanda/style_transfer/fanfanda_neuralImages/NeuralImageSynthesis/Tmp/images.hdf5',
    layers = 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1',
    gpu = 0, backend = 'cudnn', 
    output_file = '/home/fanfanda/style_transfer/fanfanda_neuralImages/NeuralImageSynthesis/Tmp/activations.hdf5'}

In [3]:
-- 初始化gpu参数
paths.dofile('Misc.lua')
-- Set gpu mode
if params.gpu >= 0 then
    require 'cutorch'
    require 'cunn'
    cutorch.setDevice(params.gpu + 1)
else
    params.backend = 'nn'
end
if params.backend == 'cudnn' then
    require 'cudnn'
    cudnn.SpatialConvolution.accGradParameters = nn.SpatialConvolutionMM.accGradParameters -- ie: nop
end

In [4]:
-- Load network from caffemodel
loadcaffe_backend = params.backend
cnn = loadcaffe.load('network', params.caffe_model, params.backend):float()
cnn = set_datatype(cnn, params.gpu)

-- Load images
local f = hdf5.open(params.images, 'r')
images = f:all()['images']
f:close()
images = set_datatype(images, params.gpu)

Successfully loaded /home/fanfanda/style_transfer/fanfanda_neuralImages/NeuralImageSynthesis/Models/VGG_ILSVRC_19_layers_conv.caffemodel


conv1_1: 64 3 3 3
conv1_2: 64 64 3 3
conv2_1: 128 64 3 3


conv2_2: 128 128 3 3
conv3_1: 256 128 3 3


conv3_2: 256 256 3 3


conv3_3: 256 256 3 3


...

conv4_4: 512 512 3 3


conv5_1: 512 512 3 3


conv5_2: 512 512 3 3


conv5_3: 512 512 3 3


conv5_4: 512 512 3 3


In [5]:
params.layers:split(",")

{
  1 : relu1_1
  2 : relu2_1
  3 : relu3_1
  4 : relu4_1
  5 : relu5_1
}


In [7]:
-- Set up new network only using necessary layers 
net = nn.Sequential()
layers = nil 
if params.layers == 'all' then
    net = cnn:clone()
    if params.reflectance then
        print('Warning, no reflectance padding for layers "all"')
    end
else
    layers = params.layers:split(",")
    local next_layer_ndx = 1
    for i = 1, #cnn do
        if next_layer_ndx <= length(layers) then
            local layer = cnn:get(i)
            local layer_name = layer.name
            local is_convolution = (layer_type == 'cudnn.SpatialConvolution' or layer_type == 'nn.SpatialConvolution')
            if is_convolution and params.reflectance then
                local padW, padH = layer.padW, layer.padH
                local pad_layer = nn.SpatialReflectionPadding(padW, padW, padH, padH)
                pad_layer = set_datatype(pad_layer, params.gpu)
                net:add(pad_layer)
                layer.padW = 0
                layer.padH = 0
            end
            net:add(layer)
            if layer_name == layers[next_layer_ndx] then
                next_layer_ndx = next_layer_ndx + 1
            end
        end
    end
end
cnn = nil
collectgarbage()

In [11]:
-- Pass images through the network
net:forward(images)

local f = hdf5.open(params.output_file, 'w')
if params.layers == 'all' then
    for i = 1, #net do
        local layer = net:get(i)
        local layer_name = layer.name
        local layer_type = torch.type(layer)
        f:write(layer_name, layer.output:double())
    end
else
    next_layer_ndx = 1
    if 'data'  == layers[next_layer_ndx] then
        f:write('data', images:double())
        next_layer_ndx = next_layer_ndx + 1
    end
    for i = 1, #net do
        if next_layer_ndx <= length(layers) then
            local layer = net:get(i)
            local layer_name = layer.name
            local layer_type = torch.type(layer)
            if layer_name == layers[next_layer_ndx] then
                f:write(layer_name, layer.output:double())
                next_layer_ndx = next_layer_ndx + 1
            end
        end
    end
end
f:close()

In [6]:
for i = 1, #cnn do
    local layer = cnn:get(i)
    local layer_name = layer.name
    print(i, layer.name)
end

1	conv1_1	
2	relu1_1	
3	conv1_2	
4	relu1_2	
5	pool1	
6	conv2_1	
7	relu2_1	
8	conv2_2	
9	relu2_2	
10	pool2	
11	conv3_1	
12	relu3_1	
13	conv3_2	
14	relu3_2	
15	conv3_3	
16	relu3_3	
17	conv3_4	
18	relu3_4	
19	pool3	
20	conv4_1	
21	relu4_1	
22	conv4_2	
23	relu4_2	
24	conv4_3	
25	relu4_3	
26	conv4_4	
27	relu4_4	
28	pool4	
29	conv5_1	
30	relu5_1	
31	conv5_2	
32	relu5_2	
33	conv5_3	
34	relu5_3	
35	conv5_4	
36	relu5_4	
37	pool5	


In [8]:
for i = 1, #net do
    local layer = net:get(i)
    local layer_name = layer.name
    print(i, layer.name)
end

1	conv1_1	
2	relu1_1	
3	conv1_2	
4	relu1_2	
5	pool1	
6	conv2_1	
7	relu2_1	
8	conv2_2	
9	relu2_2	
10	pool2	
11	conv3_1	
12	relu3_1	
13	conv3_2	
14	relu3_2	
15	conv3_3	
16	relu3_3	
17	conv3_4	
18	relu3_4	
19	pool3	
20	conv4_1	
21	relu4_1	
22	conv4_2	
23	relu4_2	
24	conv4_3	
25	relu4_3	
26	conv4_4	
27	relu4_4	
28	pool4	
29	conv5_1	
30	relu5_1	
